# HybridTwoWay Model


In [ ]:
!pip install roboflow torch torchvision torchaudio opencv-python numpy tqdm pillow matplotlib albumentations timm

In [ ]:
# ============================================
# Cell 1: 설치 및 기본 Import, Roboflow 다운로드
# ============================================

import math
import os
from typing import List, Tuple
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from roboflow import Roboflow
from tqdm import tqdm
import timm # [NEW] Pretraining용 라이브러리
import albumentations as A # [NEW] 증강 라이브러리

rf = Roboflow(api_key="HG9M6YJZpcCUgAQaKO9v")
project = rf.workspace("arakon").project("detection-base-hqaeg")
version = project.version(6)
dataset = version.download("yolov8")

print(f'Roboflow dataset downloaded to: {dataset.location}')


In [ ]:
# ============================================
# Cell 2: 기본 Conv 블록 + Stem
# ============================================

def conv_bn_act(in_ch, out_ch, k=3, s=1, p=1, act=True):
    m = [nn.Conv2d(in_ch, out_ch, k, s, p, bias=False),
         nn.BatchNorm2d(out_ch)]
    if act:
        m.append(nn.SiLU(inplace=True))
    return nn.Sequential(*m)

class FixedGaussianBlur(nn.Module):
    def __init__(self, channels, k=5, sigma=1.0):
        super().__init__()
        grid = torch.arange(k).float() - (k - 1) / 2
        gauss = torch.exp(-(grid ** 2) / (2 * sigma ** 2))
        kernel1d = gauss / gauss.sum()
        kernel2d = torch.outer(kernel1d, kernel1d)
        weight = kernel2d[None, None, :, :].repeat(channels, 1, 1, 1)
        self.register_buffer('weight', weight)
        self.groups = channels
        self.k = k

    def forward(self, x):
        pad = (self.k // 2,) * 4
        return F.conv2d(F.pad(x, pad, mode='reflect'),
                        self.weight, groups=self.groups)
class AnomalyAwareStem(nn.Module):
    def __init__(self, in_ch=3, base_ch=48):
        super().__init__()
        C1, C2, C3 = base_ch, base_ch * 2, base_ch * 4
        self.stem = nn.Sequential(
            conv_bn_act(in_ch, C1, 3, 2, 1),
            conv_bn_act(C1, C2, 3, 2, 1),
            conv_bn_act(C2, C3, 3, 2, 1),
        )
        self.blur = FixedGaussianBlur(in_ch, k=5, sigma=1.0)
        self.anom = nn.Sequential(
            nn.Conv2d(in_ch, in_ch, 3, 1, 1, groups=in_ch, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_ch, C3 // 4, 1, 1, 0, bias=False),
            nn.BatchNorm2d(C3 // 4),
            nn.SiLU(inplace=True),
        )
        self.fuse = nn.Conv2d(C3 + C3 // 4, C3, 1, 1, 0, bias=False)
        self.fuse_bn = nn.BatchNorm2d(C3)
        self.vis_head = nn.Conv2d(C3, 1, 1, 1, 0)
        self.base_ch = base_ch # 저장

    @property
    def out_channels(self):
        return 4 * self.base_ch

    def forward(self, x):
        f_main = self.stem(x)
        blurred = self.blur(x)
        high = x - blurred
        high_ds = F.interpolate(high, size=f_main.shape[-2:], mode='bilinear', align_corners=False)
        f_anom = self.anom(high_ds)
        f = torch.cat([f_main, f_anom], dim=1)
        f = self.fuse_bn(self.fuse(f))
        f = F.silu(f, inplace=True)
        v = torch.sigmoid(self.vis_head(f_main))
        return f, v


In [ ]:
# ============================================
# Cell 3: ViT Encoder + Feedback Adapter
# ============================================

class PatchEmbed1x1(nn.Module):
    def __init__(self, in_ch, embed_dim):
        super().__init__()
        self.proj = nn.Conv2d(in_ch, embed_dim, 1, 1, 0, bias=False)
        self.bn = nn.BatchNorm2d(embed_dim)

    def forward(self, x):
        x = self.proj(x)
        x = self.bn(x)
        x = F.silu(x, inplace=True)
        return x

class MLP(nn.Module):
    def __init__(self, dim, mlp_ratio=4.0, drop=0.0):
        super().__init__()
        hidden = int(dim * mlp_ratio)
        self.fc1 = nn.Linear(dim, hidden)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden, dim)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class MultiheadSelfAttention(nn.Module):
    def __init__(self, dim, num_heads=8, attn_drop=0.0, proj_drop=0.0):
        super().__init__()
        assert dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv = nn.Linear(dim, dim * 3, bias=True)
        self.attn_drop_p = attn_drop 
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = (self.qkv(x)
               .reshape(B, N, 3, self.num_heads, self.head_dim)
               .permute(2, 0, 3, 1, 4))
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        # Flash Attention
        out = F.scaled_dot_product_attention(
            q, k, v,
            dropout_p=self.attn_drop_p if self.training else 0.0,
            scale=self.scale
        )
        out = out.transpose(1, 2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, dim, num_heads=8, mlp_ratio=4.0, drop=0.0):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = MultiheadSelfAttention(dim, num_heads, drop, drop)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, mlp_ratio, drop)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

class ViTEncoder(nn.Module):
    def __init__(self, embed_dim=512, depth=8, num_heads=8):
        super().__init__()
        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads, mlp_ratio=4.0, drop=0.0)
            for _ in range(depth)
        ])

    def forward(self, tokens):
        for blk in self.blocks:
            tokens = blk(tokens)
        return tokens

class FeedbackAdapter(nn.Module):
    def __init__(self, d_token: int, c_stem: int, use_bn: bool = True):
        super().__init__()
        layers = [nn.Conv2d(d_token, c_stem * 2, 1, 1, 0, bias=not use_bn)]
        if use_bn:
            layers.append(nn.BatchNorm2d(c_stem * 2))
        layers.append(nn.SiLU(inplace=True))
        self.adapter = nn.Sequential(*layers)

    def forward(self, tokens, Ht, Wt, f_stem):
        B, N, D = tokens.shape
        t2d = tokens.transpose(1, 2).reshape(B, D, Ht, Wt)
        ab = self.adapter(t2d)
        Cs = f_stem.shape[1]
        gamma, beta = torch.split(ab, Cs, dim=1)
        return f_stem * (1 + torch.tanh(gamma)) + beta


In [ ]:
# ============================================
# Cell 4: PANLite Neck
# ============================================

class PANLite(nn.Module):
    def __init__(self, in_ch=512, mid=256):
        super().__init__()
        self.lateral = conv_bn_act(in_ch, mid, 1, 1, 0)
        self.down4 = conv_bn_act(mid, mid, 3, 2, 1)
        self.down5 = conv_bn_act(mid, mid, 3, 2, 1)
        self.up4 = conv_bn_act(mid + mid, mid, 3, 1, 1)
        self.up3 = conv_bn_act(mid + mid, mid, 3, 1, 1)
        self.down_f4 = conv_bn_act(mid, mid, 3, 2, 1)
        self.fuse4 = conv_bn_act(mid + mid, mid, 3, 1, 1)
        self.down_f5 = conv_bn_act(mid, mid, 3, 2, 1)
        self.fuse5 = conv_bn_act(mid + mid, mid, 3, 1, 1)

    def forward(self, p3):
        p3 = self.lateral(p3)
        p4 = self.down4(p3)
        p5 = self.down5(p4)
        p4u = F.interpolate(p5, size=p4.shape[-2:], mode='nearest')
        p4 = self.up4(torch.cat([p4, p4u], dim=1))
        p3u = F.interpolate(p4, size=p3.shape[-2:], mode='nearest')
        p3 = self.up3(torch.cat([p3, p3u], dim=1))
        p4b = self.down_f4(p3)
        p4 = self.fuse4(torch.cat([p4, p4b], dim=1))
        p5b = self.down_f5(p4)
        p5 = self.fuse5(torch.cat([p5, p5b], dim=1))
        return p3, p4, p5

In [ ]:
# ============================================
# Cell 5: YOLO Head
# ============================================

class YOLOHeadLite(nn.Module):
    def __init__(self, in_ch=256, num_classes=1):
        super().__init__()
        c = in_ch
        self.stem3 = conv_bn_act(c, c, 3, 1, 1)
        self.stem4 = conv_bn_act(c, c, 3, 1, 1)
        self.stem5 = conv_bn_act(c, c, 3, 1, 1)
        self.cls3 = nn.Conv2d(c, num_classes, 1, 1, 0)
        self.obj3 = nn.Conv2d(c, 1, 1, 1, 0)
        self.box3 = nn.Conv2d(c, 4, 1, 1, 0)
        nn.init.constant_(self.obj3.bias, -4.59)
        self.cls4 = nn.Conv2d(c, num_classes, 1, 1, 0)
        self.obj4 = nn.Conv2d(c, 1, 1, 1, 0)
        self.box4 = nn.Conv2d(c, 4, 1, 1, 0)
        nn.init.constant_(self.obj4.bias, -4.59)
        self.cls5 = nn.Conv2d(c, num_classes, 1, 1, 0)
        self.obj5 = nn.Conv2d(c, 1, 1, 1, 0)
        self.box5 = nn.Conv2d(c, 4, 1, 1, 0)
        nn.init.constant_(self.obj5.bias, -4.59)

    def forward_single(self, x, stem, cls, obj, box):
        f = stem(x)
        return cls(f), obj(f), box(f)

    def forward(self, p3, p4, p5):
        c3, o3, b3 = self.forward_single(p3, self.stem3, self.cls3, self.obj3, self.box3)
        c4, o4, b4 = self.forward_single(p4, self.stem4, self.cls4, self.obj4, self.box4)
        c5, o5, b5 = self.forward_single(p5, self.stem5, self.cls5, self.obj5, self.box5)
        return [(c3, o3, b3), (c4, o4, b4), (c5, o5, b5)]

In [ ]:
# ============================================
# Cell 6: HybridTwoWay Model with Timm Pretraining
# ============================================

class HybridTwoWay(nn.Module):
    def __init__(self, 
                 in_ch=3, 
                 stem_base=32, 
                 embed_dim=192, # [변경] ViT-Tiny (192) 또는 Small (384)에 맞춰야 함. 여기선 Tiny 사용
                 vit_model_name='vit_tiny_patch16_224.augreg_in21k_ft_in1k', # [NEW] Timm Model Name
                 num_classes=3, 
                 iters=1, 
                 detach_feedback=True, 
                 img_size=640):
        super().__init__()
        self.iters = iters
        self.detach_feedback = detach_feedback
        
        # 1) Stem
        self.stem = AnomalyAwareStem(in_ch=in_ch, base_ch=stem_base)
        c_stem = stem_base * 4  # e.g., 32*4 = 128
        
        # 2) ViT from Timm (Pretrained)
        # Timm 모델 로드 (Features Only)
        print(f"🔄 Loading Pretrained Weights: {vit_model_name}...")
        self.vit = timm.create_model(vit_model_name, pretrained=True, num_classes=0)
        self.vit_dim = self.vit.num_features # e.g., 192 for tiny
        
        # Stem(128) -> ViT(192) 연결을 위한 Projection
        self.to_vit = nn.Sequential(
            nn.Conv2d(c_stem, self.vit_dim, 1),
            nn.BatchNorm2d(self.vit_dim),
            nn.SiLU()
        )
        
        # ViT(192) -> Neck(256) 연결을 위한 Projection
        self.from_vit = nn.Sequential(
            nn.Conv2d(self.vit_dim, 256, 1), # Neck Input is 256
            nn.BatchNorm2d(256),
            nn.SiLU()
        )

        # Positional Embedding (Dynamic)
        # ViT Patch size = 16. Input to ViT is Stem output (stride 8).
        # Effective patch size relative to image is 8? No, stem downsamples.
        # We will treat Stem output as feature map.
        # We rely on Timm's internal PosEmbed or add our own if needed. 
        # Timm ViT expects [B, N, D].
        
        # 3) Feedback Adapter (Dimension 맞춰서 수정)
        self.feedback = FeedbackAdapter(d_token=self.vit_dim, c_stem=c_stem, use_bn=True)
        
        # 4) Neck + Head
        self.neck = PANLite(in_ch=256, mid=256) # Neck input fixed to 256 via projection
        self.head = YOLOHeadLite(in_ch=256, num_classes=num_classes)

    def forward(self, x):
        # 1) Stem
        f_stem, vis = self.stem(x) # (B, 128, H/8, W/8)
        
        # Stem -> ViT Dimension Projection
        f_vit_in = self.to_vit(f_stem) # (B, 192, H/8, W/8)
        
        # Flatten for ViT
        B, C, Ht, Wt = f_vit_in.shape
        tokens = f_vit_in.flatten(2).transpose(1, 2) # (B, N, 192)
        
        # Timm ViT Forward
        # Timm ViT expects Pos Embed inside. But our sequence length changes with img_size.
        # We need to interpolate Timm's pos_embed to match current Ht, Wt.
        # This is handled by Timm's `forward_features` if we pass grid size, but standard ViT is rigid.
        # Hack: We use the tokens as is, assuming ViT handles arbitrary length if no pos_embed check.
        # Better: Manually interpolate Timm's pos embed.
        
        if self.vit.pos_embed is not None:
            # Resize pos_embed
            # self.vit.pos_embed: (1, N_pre + 1, D)
            # We skip class token handling for simplicity as we use features
            src_pos = self.vit.pos_embed[:, self.vit.num_prefix_tokens:]
            src_N = src_pos.shape[1]
            src_h = int(src_N**0.5)
            
            pos_embed = F.interpolate(
                src_pos.reshape(1, src_h, src_h, -1).permute(0, 3, 1, 2),
                size=(Ht, Wt), mode='bicubic', align_corners=False
            ).flatten(2).transpose(1, 2)
            
            # Add to tokens
            tokens = tokens + pos_embed

        f_fb = f_stem
        preds, aux = None, None
        
        for i in range(self.iters):
            # ViT Blocks
            tokens = self.vit.blocks(tokens)
            tokens = self.vit.norm(tokens)

            # Feedback
            toks_for_fb = tokens.detach() if self.detach_feedback else tokens
            f_fb = self.feedback(toks_for_fb, Ht, Wt, f_fb) # Modifies f_stem features
            
            # Prepare for Neck
            # We need to mix f_fb (Stem domain) and tokens (ViT domain)?
            # Or just use f_fb as per original design.
            # The original design uses `patch(f_fb)` to go to Neck.
            # Here we should project f_fb (128) -> Neck (256)
            # But wait, `f_fb` is refreshed stem feature.
            
            # Re-project f_fb to ViT dim for next loop
            if i != self.iters - 1:
                 f_vit_in = self.to_vit(f_fb)
                 tokens = f_vit_in.flatten(2).transpose(1, 2) + pos_embed

            # Project f_fb to Neck
            # Original used `self.patch` (Conv 1x1). We use `to_vit` then `from_vit`?
            # Let's use a separate projection or reuse.
            # Let's project f_fb (128) -> Neck (256) directly
            # We need a layer for this. Let's use `self.to_vit` output passed through `self.from_vit` 
            # effectively 128->192->256.
            
            p3_in = self.from_vit(self.to_vit(f_fb))
            
            p3, p4, p5 = self.neck(p3_in)
            preds = self.head(p3, p4, p5)
            aux = {"P3": p3, "P4": p4, "P5": p5, "V": vis}

        return preds, aux

In [ ]:
# ============================================
# Cell 7: Dataset with Mosaic Augmentation
# ============================================

IMG_SIZE = 640

def yolo_collate_fn(batch):
    imgs = []
    targets = []
    for img, tgt in batch:
        imgs.append(img)
        targets.append(tgt)
    imgs = torch.stack(imgs, 0)
    return imgs, targets

class YoloDataset(Dataset):
    def __init__(self, root, is_train=True, mosaic_prob=0.5):
        self.img_dir = os.path.join(root, "images")
        self.label_dir = os.path.join(root, "labels")
        self.images = sorted(os.listdir(self.img_dir))
        self.is_train = is_train
        self.mosaic_prob = mosaic_prob
        self.img_size = IMG_SIZE

    def __len__(self):
        return len(self.images)

    def load_image_and_boxes(self, index):
        # 이미지 로드
        name = self.images[index]
        img_path = os.path.join(self.img_dir, name)
        img = cv2.imread(img_path)
        if img is None: raise FileNotFoundError(f"Image not found: {img_path}")
        h, w = img.shape[:2]
        
        # 라벨 로드
        label_path = os.path.join(self.label_dir, name.replace(".jpg",".txt").replace(".png",".txt"))
        boxes = []
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    # class, x_c, y_c, w, h (normalized)
                    cls, cx, cy, bw, bh = map(float, line.split())
                    # normalized -> absolute xyxy
                    x1 = (cx - bw/2) * w
                    y1 = (cy - bh/2) * h
                    x2 = (cx + bw/2) * w
                    y2 = (cy + bh/2) * h
                    boxes.append([cls, x1, y1, x2, y2])
        return img, np.array(boxes) if len(boxes) > 0 else np.zeros((0, 5))

    def load_mosaic(self, index):
        # 4장의 이미지를 합쳐서 하나로 만듦
        indices = [index] + [random.randint(0, len(self.images) - 1) for _ in range(3)]
        
        yc, xc = [int(random.uniform(-x, 2 * self.img_size + x)) for x in [-self.img_size // 2]]
        
        mosaic_img = np.full((self.img_size * 2, self.img_size * 2, 3), 114, dtype=np.uint8)
        mosaic_boxes = []

        for i, idx in enumerate(indices):
            img, boxes = self.load_image_and_boxes(idx)
            h, w = img.shape[:2]

            # Resize keeping aspect ratio
            r = self.img_size / max(h, w)
            if r != 1: 
                img = cv2.resize(img, (int(w * r), int(h * r)), interpolation=cv2.INTER_LINEAR)
            h, w = img.shape[:2]

            # Place image in mosaic
            if i == 0:  # top left
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, self.img_size * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(self.img_size * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, w, min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, self.img_size * 2), min(self.img_size * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)

            mosaic_img[y1a:y2a, x1a:x2a] = img[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            # Adjust boxes
            if len(boxes) > 0:
                boxes[:, 1] = boxes[:, 1] * r + padw
                boxes[:, 2] = boxes[:, 2] * r + padh
                boxes[:, 3] = boxes[:, 3] * r + padw
                boxes[:, 4] = boxes[:, 4] * r + padh
                mosaic_boxes.append(boxes)

        if len(mosaic_boxes) > 0:
            mosaic_boxes = np.concatenate(mosaic_boxes, 0)
            # Clip boxes
            np.clip(mosaic_boxes[:, 1:], 0, 2 * self.img_size, out=mosaic_boxes[:, 1:])
        
        # Center Crop (640x640)
        img = mosaic_img
        boxes = mosaic_boxes if len(mosaic_boxes) > 0 else np.zeros((0, 5))
        
        # Random Center Crop
        h, w = img.shape[:2]
        if h > self.img_size and w > self.img_size:
             top = random.randint(0, h - self.img_size)
             left = random.randint(0, w - self.img_size)
             img = img[top:top+self.img_size, left:left+self.img_size]
             
             if len(boxes) > 0:
                 boxes[:, 1] -= left
                 boxes[:, 2] -= top
                 boxes[:, 3] -= left
                 boxes[:, 4] -= top
                 np.clip(boxes[:, 1:], 0, self.img_size, out=boxes[:, 1:])
        
        return img, boxes

    def __getitem__(self, idx):
        # Mosaic 적용 여부 결정
        if self.is_train and random.random() < self.mosaic_prob:
            img, boxes_xyxy = self.load_mosaic(idx)
        else:
            img, boxes_xyxy = self.load_image_and_boxes(idx)
            # Resize to IMG_SIZE (Letterbox 없이 단순 리사이즈 - 기존 유지)
            img = cv2.resize(img, (self.img_size, self.img_size))
            h_orig, w_orig = img.shape[:2] # Actually it's 640, 640 now
            # boxes need to be rescaled if not mosaic (load_image_and_boxes returns absolute original)
            # But we loaded absolute. Let's simple resize boxes logic here for non-mosaic
            # Wait, load_image_and_boxes returns absolute coord of ORIGINAL image.
            # We need to re-read mostly. Simpler approach:
            
            # Re-read raw for non-mosaic to match logic perfectly
            name = self.images[idx]
            img_raw = cv2.imread(os.path.join(self.img_dir, name))
            img_raw = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
            h0, w0 = img_raw.shape[:2]
            img = cv2.resize(img_raw, (self.img_size, self.img_size))
            
            label_path = os.path.join(self.label_dir, name.replace(".jpg",".txt").replace(".png",".txt"))
            boxes_xyxy = []
            if os.path.exists(label_path):
                with open(label_path, "r") as f:
                    for line in f.readlines():
                        c, cx, cy, bw, bh = map(float, line.split())
                        # norm -> absolute 640
                        x1 = (cx - bw/2) * self.img_size
                        y1 = (cy - bh/2) * self.img_size
                        x2 = (cx + bw/2) * self.img_size
                        y2 = (cy + bh/2) * self.img_size
                        boxes_xyxy.append([c, x1, y1, x2, y2])
            boxes_xyxy = np.array(boxes_xyxy) if len(boxes_xyxy) > 0 else np.zeros((0, 5))

        # Normalize & Tensor conversion
        img = img.astype(np.float32) / 255.0
        img = torch.from_numpy(img).permute(2, 0, 1) # (3, 640, 640)
        
        # Boxes: [cls, x1, y1, x2, y2] -> [cls, cx, cy, w, h] (Normalized) for Loss
        targets = []
        if len(boxes_xyxy) > 0:
            # Remove small boxes
            boxes_xyxy = boxes_xyxy[np.logical_and(boxes_xyxy[:, 3] > boxes_xyxy[:, 1], boxes_xyxy[:, 4] > boxes_xyxy[:, 2])]
            
            for box in boxes_xyxy:
                cls = box[0]
                x1, y1, x2, y2 = box[1:]
                cx = (x1 + x2) / 2 / self.img_size
                cy = (y1 + y2) / 2 / self.img_size
                w = (x2 - x1) / self.img_size
                h = (y2 - y1) / self.img_size
                targets.append([cls, cx, cy, w, h])
        
        targets = torch.tensor(targets, dtype=torch.float32)
        return img, targets

In [ ]:
# ============================================
# Cell 8: Task Aligned Loss (TAL) - YOLOv8 Style
# ============================================

def select_highest_overlaps(mask_pos, overlaps, n_max_boxes):
    # GT와 가장 많이 겹치는 앵커 하나는 무조건 포함시키는 로직 (안정성)
    fg_mask = mask_pos.sum(-2) > 0
    if fg_mask.all():
        return mask_pos
    _, argmax_overlaps = overlaps.max(dim=0)
    mask_pos[argmax_overlaps, torch.arange(mask_pos.shape[1], device=mask_pos.device)] = True
    return mask_pos

def select_candidates_in_gts(xy_centers, gt_bboxes, eps=1e-9):
    # 앵커 포인트가 GT 박스 안에 들어오는지 확인
    n_anchors = xy_centers.shape[0]
    bs, n_boxes, _ = gt_bboxes.shape
    lt, rb = gt_bboxes.view(-1, 1, 4).chunk(2, 2) 
    bbox_deltas = torch.cat((xy_centers[None] - lt, rb - xy_centers[None]), dim=2).view(bs, n_boxes, n_anchors, -1)
    return bbox_deltas.min(3)[0] > eps

class TaskAlignedAssigner(nn.Module):
    def __init__(self, topk=13, alpha=1.0, beta=6.0, eps=1e-9):
        super().__init__()
        self.topk = topk
        self.alpha = alpha
        self.beta = beta
        self.eps = eps

    @torch.no_grad()
    def forward(self, pd_scores, pd_bboxes, anc_points, gt_labels, gt_bboxes, mask_gt):
        # pd_scores: (B, A, C)
        # pd_bboxes: (B, A, 4)
        # anc_points: (A, 2)
        # gt_bboxes: (B, M, 4)
        
        bs, n_max_boxes = gt_bboxes.shape[:2]
        mask_pos, align_metric, overlaps = self.get_pos_mask(
            pd_scores, pd_bboxes, gt_labels, gt_bboxes, anc_points, mask_gt
        )
        
        target_gt_idx, fg_mask, mask_pos = self.select_topk_candidates(
            align_metric * mask_pos, topk_mask=mask_gt.repeat([1, 1, self.topk]).bool()
        )
        
        # Assign Targets
        target_labels = gt_labels.long().flatten()[target_gt_idx.flatten()]
        target_labels = target_labels.view(bs, -1)
        target_bboxes = gt_bboxes.view(-1, 4)[target_gt_idx.flatten()]
        target_bboxes = target_bboxes.view(bs, -1, 4)
        
        # Metric as score target
        align_metric *= mask_pos
        pos_align_metrics = align_metric.max(1)[0]
        target_scores = pos_align_metrics[..., None].repeat(1, 1, 1) # dummy expand
        
        # One-hot encoding
        t_scores = torch.zeros_like(pd_scores)
        t_scores[fg_mask] = 0.0 # Clear
        # Indexing is tricky, simple loop or scatter
        # TAL usually sets target score = metric
        
        # Simplification for Colab:
        # Just return masks and targets for loss calc
        return target_labels, target_bboxes, target_scores, fg_mask, target_gt_idx

    def get_pos_mask(self, pd_scores, pd_bboxes, gt_labels, gt_bboxes, anc_points, mask_gt):
        bs, n_max_boxes = gt_bboxes.shape[:2]
        mask_in_gts = select_candidates_in_gts(anc_points, gt_bboxes) # (B, M, A)
        
        # Compute IoU & Metric
        # pd_bboxes: (B, A, 4), gt_bboxes: (B, M, 4)
        # Expand for broadcasting
        # overlaps: (B, M, A)
        
        # Simple IoU calculation for assignment
        # ... (Full CIoU implementation omitted for brevity, using simplified bbox_iou)
        # Assuming pd_bboxes and gt_bboxes are xyxy
        
        # Broadcasting IoU
        # This can be heavy. Loop per batch if OOM.
        overlaps = []
        for b in range(bs):
            iou = box_iou(gt_bboxes[b], pd_bboxes[b]) # (M, A)
            overlaps.append(iou)
        overlaps = torch.stack(overlaps)
        
        # Metric: s^alpha * u^beta
        # pd_scores: (B, A, C) -> gather gt labels
        # pd_scores for corresponding GT class
        # gather: (B, M, A)
        bbox_scores = []
        for b in range(bs):
            # gt_labels[b]: (M, 1)
            labels = gt_labels[b].long().squeeze(-1)
            scores = pd_scores[b, :, labels].T # (M, A)
            bbox_scores.append(scores)
        bbox_scores = torch.stack(bbox_scores)
        
        align_metric = bbox_scores.pow(self.alpha) * overlaps.pow(self.beta)
        
        mask_pos = mask_in_gts * mask_gt.view(bs, n_max_boxes, 1)
        return mask_pos, align_metric, overlaps

    def select_topk_candidates(self, metrics, topk_mask=None):
        # metrics: (B, M, A)
        num_anchors = metrics.shape[-1]
        topk_metrics, topk_idxs = torch.topk(metrics, self.topk, dim=-1, largest=True)
        
        topk_mask = torch.zeros_like(metrics, dtype=torch.bool)
        topk_mask.scatter_(-1, topk_idxs, True)
        
        # Fail safe: at least one anchor
        # (Omitted)
        
        mask_pos = topk_mask
        fg_mask = mask_pos.sum(-2) > 0 # (B, A)
        
        # If anchor assigned to multiple GTs, pick max metric
        target_gt_idx = mask_pos.float().argmax(-2) # (B, A)
        
        return target_gt_idx, fg_mask, mask_pos


# ----- Loss Wrapper -----
class ComputeLoss(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        self.assigner = TaskAlignedAssigner(topk=10)
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
        self.num_classes = num_classes

    def forward(self, preds, batch_targets, img_size):
        # preds: [(cls, obj, box), ...] (3 scales)
        # batch_targets: List[Tensor] len B, Tensor(M, 5)
        
        # 1. Concat Predictions
        # We need to flatten all scales to (B, Total_Anchors, C/4)
        cls_preds = []
        box_preds = []
        anchors = []
        strides = []
        
        for i, (cls, obj, box) in enumerate(preds):
            B, C, H, W = cls.shape
            stride = img_size // H
            
            # Create Grid
            grid_y, grid_x = torch.meshgrid(torch.arange(H), torch.arange(W), indexing='ij')
            grid = torch.stack((grid_x, grid_y), 2).to(cls.device).float()
            grid = (grid + 0.5) * stride # Center
            anchors.append(grid.view(-1, 2))
            strides.append(torch.full((H*W,), stride, device=cls.device))
            
            cls_preds.append(cls.permute(0, 2, 3, 1).reshape(B, -1, C))
            # Box: xywh logits -> xyxy
            # Our model outputs xywh normalized (sigmoid). 
            # We need to decode it to absolute xyxy for Assigner
            b_p = box.permute(0, 2, 3, 1).reshape(B, -1, 4).sigmoid()
            # decode: cx, cy, w, h (norm) -> abs
            # x = (pred_x * stride + grid_x) ? No, our model predicts global norm coordinates directly.
            # Let's assume model outputs global normalized sigmoid as before.
            
            cx = b_p[..., 0] * img_size
            cy = b_p[..., 1] * img_size
            w  = b_p[..., 2] * img_size
            h  = b_p[..., 3] * img_size
            x1 = cx - w/2
            y1 = cy - h/2
            x2 = cx + w/2
            y2 = cy + h/2
            box_preds.append(torch.stack([x1, y1, x2, y2], -1))

        cls_preds = torch.cat(cls_preds, 1) # (B, A_tot, C)
        box_preds = torch.cat(box_preds, 1) # (B, A_tot, 4)
        anchors = torch.cat(anchors, 0)     # (A_tot, 2)
        
        # 2. Prepare Targets
        # Pad targets to fixed size M for batch processing
        max_boxes = max([len(t) for t in batch_targets])
        batch_gt_labels = torch.zeros(B, max_boxes, 1).to(cls_preds.device)
        batch_gt_bboxes = torch.zeros(B, max_boxes, 4).to(cls_preds.device)
        batch_mask_gt = torch.zeros(B, max_boxes).to(cls_preds.device)
        
        for b, t in enumerate(batch_targets):
            n = len(t)
            if n > 0:
                batch_gt_labels[b, :n, 0] = t[:, 0]
                # t is [cls, cx, cy, w, h] norm
                cx, cy, w, h = t[:, 1], t[:, 2], t[:, 3], t[:, 4]
                x1 = (cx - w/2) * img_size
                y1 = (cy - h/2) * img_size
                x2 = (cx + w/2) * img_size
                y2 = (cy + h/2) * img_size
                batch_gt_bboxes[b, :n] = torch.stack([x1, y1, x2, y2], -1)
                batch_mask_gt[b, :n] = 1.0

        # 3. Assign
        if max_boxes == 0:
            return cls_preds.sum() * 0.0 # Zero loss
            
        targets = self.assigner(
            cls_preds.sigmoid(), box_preds, anchors, 
            batch_gt_labels, batch_gt_bboxes, batch_mask_gt
        )
        target_labels, target_bboxes, target_scores, fg_mask, target_gt_idx = targets
        
        # 4. Calculate Loss
        # Classification (BCE)
        # target_scores: (B, M, A) -> max over M -> (B, A) is essentially "soft label"
        # But Assigner output is tricky.
        # Simplified:
        # target_labels: (B, A) -> Class index
        # fg_mask: (B, A) -> Positive mask
        
        # Build target one-hot scaled by alignment metric (Soft Label)
        # For simplicity, Hard Label 1.0 for positives
        
        target_cls = torch.zeros_like(cls_preds)
        # Scatter indices
        # This part needs careful indexing.
        # Using simple BCE on positives for now
        
        num_pos = fg_mask.sum().clamp(1.0)
        
        # Cls Loss
        # For positives: Target = 1 (or IoU score). For negatives: Target = 0
        target_cls[fg_mask] = 0 
        # We need to set 1 at target_labels indices
        for b in range(B):
            if fg_mask[b].sum() > 0:
                pos_idx = fg_mask[b].nonzero().squeeze(-1)
                cls_idx = target_labels[b][pos_idx].long()
                target_cls[b, pos_idx, cls_idx] = 1.0 # Can be replaced with IoU score
        
        loss_cls = self.bce(cls_preds, target_cls).sum() / num_pos
        
        # Box Loss (CIoU / GIoU)
        loss_box = 0.0
        if fg_mask.sum() > 0:
            pos_pred_box = box_preds[fg_mask]
            pos_tgt_box = target_bboxes[fg_mask]
            loss_box = (1.0 - box_iou(pos_pred_box, pos_tgt_box).diag()).sum() / num_pos
            
        return loss_cls + 4.0 * loss_box

# Replace the old function with this class
criterion = ComputeLoss(num_classes=3)

def yolo_loss_tal(preds, targets, img_size):
    return criterion(preds, targets, img_size)

In [ ]:
# ============================================
# Cell 9: Decode Predictions + mAP Evaluation
# ============================================

def box_iou(box1, box2):
    N = box1.size(0)
    M = box2.size(0)
    if N == 0 or M == 0: return torch.zeros(N, M)
    tl = torch.max(box1[:, None, :2], box2[:, :2])
    br = torch.min(box1[:, None, 2:], box2[:, 2:])
    wh = (br - tl).clamp(min=0)
    inter = wh[..., 0] * wh[..., 1]
    area1 = (box1[:, 2]-box1[:, 0]) * (box1[:, 3]-box1[:, 1])
    area2 = (box2[:, 2]-box2[:, 0]) * (box2[:, 3]-box2[:, 1])
    return inter / (area1[:, None] + area2 - inter + 1e-6)

def nms(boxes, scores, iou_thres=0.5):
    if boxes.numel() == 0: return torch.zeros(0, dtype=torch.long, device=boxes.device)
    idxs = scores.argsort(descending=True)
    keep = []
    while idxs.numel() > 0:
        i = idxs[0]
        keep.append(i.item())
        if idxs.numel() == 1: break
        ious = box_iou(boxes[i].unsqueeze(0), boxes[idxs[1:]]).squeeze(0)
        idxs = idxs[1:][ious < iou_thres]
    return torch.tensor(keep, dtype=torch.long, device=boxes.device)

def decode_predictions(preds, img_size=512, conf_thres=0.25, nms_iou_thres=0.5):
    all_outputs = []
    B = preds[0][0].shape[0]
    
    for b in range(B):
        dets_all = []
        for (cls_pred, obj_pred, box_pred) in preds:
            B_s, C, H, W = cls_pred.shape
            
            cls_logits = cls_pred[b].permute(1,2,0).reshape(H*W, C)
            box_logits = box_pred[b].permute(1,2,0).reshape(H*W, 4)
            
            # [변경] TAL에서는 Objectness를 별도로 쓰지 않음 (학습 안 됨)
            # 따라서 cls_scores 자체가 신뢰도가 됨
            cls_scores = cls_logits.sigmoid()
            box_norm = box_logits.sigmoid()
            
            cls_max_scores, cls_ids = cls_scores.max(dim=-1)
            
            # [변경] scores = obj_scores * cls_max_scores  <-- 삭제
            scores = cls_max_scores  # <-- Class score만 사용
            
            mask = scores > conf_thres
            if mask.sum() == 0: continue
            
            scores_ = scores[mask]
            cls_ids_ = cls_ids[mask]
            boxes = box_norm[mask]
            
            x_c, y_c, w, h = boxes[:, 0]*img_size, boxes[:, 1]*img_size, boxes[:, 2]*img_size, boxes[:, 3]*img_size
            x1, y1 = (x_c - w/2).clamp(0, img_size), (y_c - h/2).clamp(0, img_size)
            x2, y2 = (x_c + w/2).clamp(0, img_size), (y_c + h/2).clamp(0, img_size)
            
            boxes_xyxy = torch.stack([x1, y1, x2, y2], dim=1)
            
            keep = nms(boxes_xyxy, scores_, iou_thres=nms_iou_thres)
            if keep.numel() == 0: continue
            
            dets = torch.cat([boxes_xyxy[keep], scores_[keep].unsqueeze(1), cls_ids_[keep].float().unsqueeze(1)], dim=1)
            dets_all.append(dets)
            
        all_outputs.append(torch.cat(dets_all, dim=0) if len(dets_all) > 0 else [])
    return all_outputs

def compute_ap(recall, precision):
    mrec = torch.cat([torch.tensor([0.0]), recall, torch.tensor([1.0])])
    mpre = torch.cat([torch.tensor([0.0]), precision, torch.tensor([0.0])])
    for i in range(mpre.size(0)-1, 0, -1):
        mpre[i-1] = torch.max(mpre[i-1], mpre[i])
    idx = (mrec[1:] != mrec[:-1]).nonzero().squeeze()
    return ((mrec[idx+1] - mrec[idx]) * mpre[idx+1]).sum().item()

def evaluate_map(model, dataloader, num_classes=3, img_size=512, iou_thr=0.5, conf_thres=0.25):
    model.eval()
    device = next(model.parameters()).device
    all_dets = {c: [] for c in range(num_classes)}
    all_gts  = {c: [] for c in range(num_classes)}
    global_img_id = 0
    
    with torch.no_grad():
        for batch_i, (imgs, targets) in enumerate(dataloader):
            imgs = imgs.to(device)
            targets = [t.to(device) for t in targets]
            preds, _ = model(imgs)
            dets_list = decode_predictions(preds, img_size=img_size, conf_thres=conf_thres)
            
            for b in range(len(imgs)):
                dets = dets_list[b]
                gt = targets[b]
                current_img_id = global_img_id
                global_img_id += 1
                
                if len(gt) > 0:
                    gcls = gt[:, 0].long()
                    gxy, gwh = gt[:, 1:3] * img_size, gt[:, 3:5] * img_size
                    gx1, gy1 = gxy[:, 0] - gwh[:, 0]/2, gxy[:, 1] - gwh[:, 1]/2
                    gx2, gy2 = gxy[:, 0] + gwh[:, 0]/2, gxy[:, 1] + gwh[:, 1]/2
                    gboxes = torch.stack([gx1, gy1, gx2, gy2], dim=1)
                    for c in range(num_classes):
                        mask = (gcls == c)
                        if mask.sum() > 0: all_gts[c].append((current_img_id, gboxes[mask].cpu()))
                
                if dets is not None and len(dets) > 0:
                    boxes, scores, cls_ids = dets[:, :4], dets[:, 4], dets[:, 5].long()
                    for c in range(num_classes):
                        mask = (cls_ids == c)
                        if mask.sum() > 0: all_dets[c].append((current_img_id, scores[mask].cpu(), boxes[mask].cpu()))

    aps = []
    for c in range(num_classes):
        gts_c = all_gts[c]
        if len(gts_c) == 0: continue
        n_gt = sum(boxes.size(0) for _, boxes in gts_c)
        gt_dict = {}
        for img_id, boxes in gts_c:
            gt_dict.setdefault(img_id, [])
            gt_dict[img_id].append({"boxes": boxes, "matched": torch.zeros(boxes.size(0), dtype=torch.bool)})
        
        dets_c = all_dets[c]
        if len(dets_c) == 0:
            aps.append(0.0)
            continue
        
        scores_all, boxes_all, img_ids_all = [], [], []
        for img_id, scores, boxes in dets_c:
            for i in range(boxes.size(0)):
                scores_all.append(scores[i].item())
                boxes_all.append(boxes[i])
                img_ids_all.append(img_id)
        
        scores_all = torch.tensor(scores_all)
        boxes_all = torch.stack(boxes_all, dim=0)
        order = scores_all.argsort(descending=True)
        scores_all, boxes_all = scores_all[order], boxes_all[order]
        img_ids_all = [img_ids_all[i] for i in order]
        
        tps, fps = torch.zeros(len(scores_all)), torch.zeros(len(scores_all))
        for i in range(len(scores_all)):
            img_id = img_ids_all[i]
            pred_box = boxes_all[i].unsqueeze(0)
            if img_id not in gt_dict:
                fps[i] = 1; continue
            gt_entry = gt_dict[img_id][0]
            ious = box_iou(pred_box, gt_entry["boxes"]).squeeze(0)
            if ious.numel() == 0: fps[i] = 1; continue
            max_iou, max_idx = ious.max(0)
            if max_iou >= iou_thr and not gt_entry["matched"][max_idx]:
                tps[i] = 1; gt_entry["matched"][max_idx] = True
            else: fps[i] = 1
        
        tp_cum, fp_cum = torch.cumsum(tps, dim=0), torch.cumsum(fps, dim=0)
        recall = tp_cum / (n_gt + 1e-6)
        precision = tp_cum / (tp_cum + fp_cum + 1e-6)
        aps.append(compute_ap(recall, precision))
    
    mAP = sum(aps) / len(aps) if len(aps) > 0 else 0.0
    return mAP, aps

In [ ]:
DATA_PATH = dataset.location
train_dataset = YoloDataset(os.path.join(DATA_PATH, "train"))
val_dataset   = YoloDataset(os.path.join(DATA_PATH, "valid"))
test_dataset  = YoloDataset(os.path.join(DATA_PATH, "test"))
# [최적화] num_workers=2, pin_memory=True 추가
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=yolo_collate_fn, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, collate_fn=yolo_collate_fn, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=yolo_collate_fn, num_workers=2, pin_memory=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

In [ ]:
# ============================================
# Cell 10: 학습 준비
# ============================================
cfg = dict(
    in_ch=3,
    stem_base=64,
    embed_dim=192,   # ViT Tiny 기준
    vit_depth=4,
    vit_heads=8,     # Tiny면 보통 3~6인데 8도 OK
    num_classes=3,
    iters=2,
    detach_feedback=False,
    img_size=IMG_SIZE
)

model = HybridTwoWay(**cfg).to(device)

# [선택] torch.compile (PyTorch 2.0+, Colab T4/L4)
try:
    model = torch.compile(model)
    print("✅ Model compiled with torch.compile")
except:
    print("⚠️ torch.compile failed/skipped")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scaler = torch.amp.GradScaler('cuda')
EPOCHS = 5 # 조정 필요

best_map = -1.0
best_epoch = -1

In [ ]:
# VRAM 사용량 체크 보조 Cell
imgs, targets = next(iter(train_loader))
imgs = imgs.to(device)
targets = [t.to(device) for t in targets]

optimizer.zero_grad()
with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
    preds, aux = model(imgs)
    loss = yolo_loss(preds, targets, img_size=IMG_SIZE)

loss.backward()

print("Max allocated:", torch.cuda.max_memory_allocated() / 1024**3, "GB")
print("Max reserved :", torch.cuda.max_memory_reserved() / 1024**3, "GB")


In [ ]:
# ============================================
# Cell 11: 학습 루프 (AMP + Flash Attention + PosEmbed + TAL Loss)
# ============================================

print("🚀 Start Training...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0.0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for imgs, targets in loop:
        imgs = imgs.to(device)
        targets = [t.to(device) for t in targets]
        optimizer.zero_grad()

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            preds, aux = model(imgs)
            
            # [수정됨] 여기서 yolo_loss -> yolo_loss_tal 로 변경해야 최신 로직이 적용됩니다!
            loss = yolo_loss_tal(preds, targets, img_size=IMG_SIZE)

        scaler.scale(loss).backward()
        
        # [안전장치] 그래디언트 클리핑 (detach_feedback=False일 때 필수)
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        loop.set_postfix(loss=f"{loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} | Train Average Loss: {avg_loss:.4f}")
    
    # Validation
    # 검증 시에도 iou_thr 등을 조금 더 엄격하게 보거나 유지해도 됩니다.
    val_map, val_aps = evaluate_map(model, val_loader, num_classes=cfg["num_classes"], img_size=IMG_SIZE, iou_thr=0.5, conf_thres=0.001)
    print(f"Epoch {epoch+1} | Val mAP@0.5: {val_map:.4f}")

    if val_map > best_map:
        best_map = val_map
        best_epoch = epoch + 1
        # compile 사용 시 원본 state_dict 저장을 위해 _orig_mod 확인
        state_dict = model._orig_mod.state_dict() if hasattr(model, '_orig_mod') else model.state_dict()
        ckpt = {"state_dict": state_dict, "cfg": cfg, "epoch": best_epoch, "val_map": best_map}
        torch.save(ckpt, "hybrid_two_way_best.pt")
        print(f"✅ Best model saved! (Val mAP: {best_map:.4f})")

print("🏁 Training Finished!")

In [ ]:
# ============================================
# Cell 12: 저장된 Best 모델 불러오기 및 테스트 평가
# ============================================

# 저장된 체크포인트 로드
checkpoint = torch.load("hybrid_two_way_best.pt", map_location=device)
loaded_cfg = checkpoint["cfg"]

print(f"📄 Loaded Config: {loaded_cfg}")

# 모델 초기화 (저장된 Config대로 복원)
model = HybridTwoWay(**loaded_cfg).to(device)
model.load_state_dict(checkpoint["state_dict"])
model.eval()

print(f"✅ Model Loaded from Epoch {checkpoint['epoch']} (Val mAP: {checkpoint['val_map']:.4f})")

# 테스트셋 평가
# TAL 방식은 score가 낮게 깔릴 수 있으므로 conf_thres를 낮춰서(0.001) mAP를 측정하는 게 국룰입니다.
test_map, class_aps = evaluate_map(model, test_loader, num_classes=loaded_cfg["num_classes"], img_size=IMG_SIZE, conf_thres=0.001)

print(f"\n🏆 Final Test mAP@0.5: {test_map:.4f}")
for i, ap in enumerate(class_aps):
    print(f"   Class {i} AP@0.5: {ap:.4f}")

In [ ]:
# ============================================
# Cell 13: Quick Sanity Check (입출력 shape 확인)
# ============================================

x = torch.randn(2, 3, 640, 640).to(device)
preds, aux = model(x)

for level, (c, o, b) in zip(["P3","P4","P5"], preds):
    print(f"[{level}] cls: {list(c.shape)}, obj: {list(o.shape)}, box: {list(b.shape)}")


In [ ]:
# 데이터 분포 확인
# 0 : APC, 1 : Tank, 2 : Person

from collections import Counter

def count_class_dist(dataset, name):
    cnt = Counter()
    for i in range(len(dataset)):
        _, tgt = dataset[i]
        if tgt.numel() == 0:
            continue
        cls_ids = tgt[:, 0].long().tolist()
        cnt.update(cls_ids)
    print(f"\n[{name}] class distribution:")
    for k in sorted(cnt.keys()):
        print(f"  class {k}: {cnt[k]} boxes")

count_class_dist(train_dataset, "train")
count_class_dist(val_dataset,   "val")
count_class_dist(test_dataset,  "test")